## Case Study Continued

In [120]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LinearRegression
from scipy.stats import boxcox
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
pd.options.display.max_rows = 50

<b> Clean data and remove null values

In [121]:
new_df = pd.read_csv('Data/file4.csv')
new_df = new_df.drop("Unnamed: 0", axis=1)

In [122]:
new_df = new_df.drop(["Customer", "Vehicle Type", "Effective To Date"], axis=1)

In [123]:
new_df.isna().sum()

State                            631
Customer Lifetime Value            0
Response                         631
Coverage                           0
Education                          0
EmploymentStatus                   0
Gender                             0
Income                             0
Location Code                      0
Marital Status                     0
Monthly Premium Auto               0
Months Since Last Claim          633
Months Since Policy Inception      0
Number of Open Complaints        633
Number of Policies                 0
Policy Type                        0
Policy                             0
Renew Offer Type                   0
Sales Channel                      0
Total Claim Amount                 0
Vehicle Class                    622
Vehicle Size                     622
dtype: int64

In [124]:
new_df["Response"] = new_df["Response"].fillna("No")

In [125]:
new_df["Number of Open Complaints"] = new_df["Number of Open Complaints"].fillna(0.0)


In [126]:
new_df = new_df.dropna()

In [127]:
new_df.isna().sum()

State                            0
Customer Lifetime Value          0
Response                         0
Coverage                         0
Education                        0
EmploymentStatus                 0
Gender                           0
Income                           0
Location Code                    0
Marital Status                   0
Monthly Premium Auto             0
Months Since Last Claim          0
Months Since Policy Inception    0
Number of Open Complaints        0
Number of Policies               0
Policy Type                      0
Policy                           0
Renew Offer Type                 0
Sales Channel                    0
Total Claim Amount               0
Vehicle Class                    0
Vehicle Size                     0
dtype: int64

<b> Split into dependant and independant variables

In [128]:
x=new_df.drop("Total Claim Amount", axis=1)
y=new_df["Total Claim Amount"]

In [129]:
xn = x._get_numeric_data()
xn

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies
0,4809.216960,48029,61,7.0,52,0.0,9
1,2228.525238,0,64,3.0,26,0.0,1
2,14947.917300,22139,100,34.0,31,0.0,2
3,22332.439460,49078,97,10.0,3,0.0,2
6,5035.035257,37405,63,8.0,99,3.0,4
...,...,...,...,...,...,...,...
10903,2771.663013,59855,74,30.0,82,4.0,1
10904,19228.463620,0,187,14.0,32,0.0,2
10906,5259.444853,61146,65,7.0,68,0.0,6
10907,23893.304100,39837,201,11.0,63,0.0,2


In [154]:
xc = x.select_dtypes(include=['object'])
xc = pd.get_dummies(xc)
xc

,State_Arizona,State_California,State_Nevada,State_Oregon,State_Washington,Response_No,Response_Yes,Coverage_Basic,Coverage_Extended,Coverage_Premium,...,Sales Channel_Web,Vehicle Class_Four-Door Car,Vehicle Class_Luxury Car,Vehicle Class_Luxury SUV,Vehicle Class_SUV,Vehicle Class_Sports Car,Vehicle Class_Two-Door Car,Vehicle Size_Large,Vehicle Size_Medsize,Vehicle Size_Small
0,1,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
1,0,1,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
2,0,0,0,0,1,1,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
3,0,0,0,1,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,1,0
6,0,1,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10903,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
10904,0,0,1,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
10906,0,0,0,1,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
10907,1,0,0,0,0,1,0,0,1,0,...,1,0,0,1,0,0,0,0,1,0


In [156]:
x = pd.concat([xn, xc], axis=1)
x

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,State_Arizona,State_California,State_Nevada,...,Sales Channel_Web,Vehicle Class_Four-Door Car,Vehicle Class_Luxury Car,Vehicle Class_Luxury SUV,Vehicle Class_SUV,Vehicle Class_Sports Car,Vehicle Class_Two-Door Car,Vehicle Size_Large,Vehicle Size_Medsize,Vehicle Size_Small
0,4809.216960,48029,61,7.0,52,0.0,9,1,0,0,...,0,1,0,0,0,0,0,0,1,0
1,2228.525238,0,64,3.0,26,0.0,1,0,1,0,...,0,1,0,0,0,0,0,0,1,0
2,14947.917300,22139,100,34.0,31,0.0,2,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,22332.439460,49078,97,10.0,3,0.0,2,0,0,0,...,0,1,0,0,0,0,0,0,1,0
6,5035.035257,37405,63,8.0,99,3.0,4,0,1,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10903,2771.663013,59855,74,30.0,82,4.0,1,1,0,0,...,0,0,0,0,0,0,1,0,1,0
10904,19228.463620,0,187,14.0,32,0.0,2,0,0,1,...,0,0,0,1,0,0,0,0,1,0
10906,5259.444853,61146,65,7.0,68,0.0,6,0,0,0,...,0,1,0,0,0,0,0,0,1,0
10907,23893.304100,39837,201,11.0,63,0.0,2,1,0,0,...,1,0,0,1,0,0,0,0,1,0


In [158]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

<b> Linear Regression

In [159]:
reg = LinearRegression().fit(x_train, y_train)

In [160]:
reg.score(x_train, y_train)

0.7718211784216935

In [162]:
reg.coef_

array([ 3.91123872e-05, -2.38317452e-04,  4.93704839e+00,  3.16382752e-01,
        3.67406926e-02, -1.94644735e+00,  7.67407866e-01, -1.87683155e+00,
        6.96898348e-01,  3.78620075e+00, -5.12887087e+00,  2.52260333e+00,
        1.05605241e+01, -1.05605241e+01,  4.26729685e+00, -5.35847261e+00,
        1.09117576e+00,  3.13715622e+00,  8.86882584e-02, -1.16439567e+01,
        1.40987790e+01, -5.68066669e+00, -1.44328236e+01, -2.54236959e+01,
        6.03374307e+00, -1.90003526e+01,  5.28231291e+01, -6.63040229e+00,
        6.63040229e+00, -1.99454173e+02,  1.78486378e+02,  2.09677951e+01,
       -1.86161332e+01, -2.40427675e+01,  4.26589007e+01, -5.16338422e+00,
       -1.95262421e+00,  7.11600842e+00, -4.51145794e+00,  2.07354958e-01,
       -8.59281235e-01,  3.19969002e+00, -4.07315174e+00, -1.07916249e+00,
        3.49617297e+01, -9.27219098e+00, -1.85735303e+01, -9.85682466e+00,
        1.25997328e+01, -1.92601564e-01, -2.55030661e+00,  1.28126377e-01,
        2.12801815e+00, -

In [163]:
reg.intercept_

-86.0828726323428

<b> Model Validation

In [164]:
y_pred = reg.predict(x_test)

In [165]:
MSE = mse(y_test, y_pred)
MSE

19804.06147872557

In [166]:
RMSE = mse(y_test, y_pred, squared=False)
RMSE

140.72690389092475

In [167]:
MAE = mae(y_test, y_pred)
MAE

96.80672654639947

In [168]:
R2 = r2_score(y_test, y_pred)
R2

0.7734545901819713